In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('./data/sampled_2018.csv')

In [12]:
df.head()['categories']

0    ["Dining & Entertaining", "Dinnerware & Servew...
1    ["Dining & Entertaining", "Dinnerware & Servew...
2              ["Home & Kitchen", "Bath", "Bath Rugs"]
3              ["Home & Kitchen", "Bath", "Bath Rugs"]
4              ["Home & Kitchen", "Bath", "Bath Rugs"]
Name: categories, dtype: object

In [13]:
df.columns

Index(['rating', 'title_x', 'text', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase', 'month',
       'main_category', 'title_y', 'average_rating', 'rating_number',
       'features', 'description', 'price', 'store', 'categories', 'brand',
       'material', 'color', 'capacity', 'style', 'pattern',
       'care_instructions', 'unit_count', 'dimensions', 'num_items',
       'item_weight', 'best_sellers_rank', 'discontinued',
       'date_first_available'],
      dtype='object')

In [14]:
df[['rating', 'title_x', 'text','timestamp','helpful_vote','verified_purchase','categories','price','parent_asin','features','care_instructions']]#['features'][3]

,rating,title_x,text,timestamp,helpful_vote,verified_purchase,categories,price,parent_asin,features,care_instructions
0,5.0,"Very nice set, is very temperamental but made ...",Really nice and cute. I bought these to put on...,2018-01-12 23:07:11.420,19,1,"[""Dining & Entertaining"", ""Dinnerware & Servew...",29.99,B00M2IAPXK,"[""Condiment and dipping bowl set includes beau...",NaN
1,5.0,Really like it,"I really like it, always receive good comments...",2018-12-15 20:13:58.764,0,1,"[""Dining & Entertaining"", ""Dinnerware & Servew...",19.82,B01BGQ82LU,"[""Arch de dip chip & dip bowl"", ""Party ready c...",Hand Wash Only
2,1.0,Frayed material the first week! Not a good pro...,Junk it started to fray the same week I bought...,2018-01-29 19:39:33.495,0,1,"[""Home & Kitchen"", ""Bath"", ""Bath Rugs""]",14.96,B018CWUO3K,"[""Skid resistant: safe & reliable pvc construc...",NaN
3,1.0,Small over priced crappy product. Would never buy,Small over priced crappy product. Would never ...,2018-01-21 21:42:14.843,2,1,"[""Home & Kitchen"", ""Bath"", ""Bath Rugs""]",19.34,B09K4WX5ND,"[""Fuzzy Microfiber"", ""Imported"", ""SUPER SOFT: ...",NaN
4,1.0,One Star,Returned,2018-02-07 18:14:13.772,0,1,"[""Home & Kitchen"", ""Bath"", ""Bath Rugs""]",NaN,B00LRYF0XA,"[""30\""X46\"" Size Bath Rug Perfect for Shower, ...",Machine Wash
...,...,...,...,...,...,...,...,...,...,...,...
51930,5.0,Easy to install and great,This took just a few minutes to install and ma...,2018-12-26 05:27:32.269,0,1,[],62.95,B00N109ZPM,"[""Dedicated smoke mode with user-controlled fe...",NaN
51931,5.0,Bigger than expected,These were larger capacity than expected. Won’...,2018-12-15 16:29:55.529,0,1,[],NaN,B06W2K6XW4,[],Hand Wash
51932,5.0,turned out well,I liked the fact I could turn them around also...,2018-12-03 20:57:58.357,0,1,[],NaN,B07C3NWYSS,[],NaN
51933,5.0,Cute,My toddler doesn't go to sleep without this pi...,2018-12-04 10:45:26.514,0,1,[],26.45,B06XBG6V51,[],Hand Wash Only


In [15]:
df[['care_instructions']].value_counts()

care_instructions                     
Machine Wash                              4727
Hand Wash Only                            2336
Dishwasher Safe                           1026
Hand Wash                                  804
Machine Wash, Hand Wash Only               761
                                          ... 
dishwasher_safe, Hand Wash Only              1
do_not_bleach, cold wash, tumble dry         1
freezer-safe, Dishwasher Safe                1
machine washable on gentle, cold cycle       1
Wipe with damp wash cloth                    1
Name: count, Length: 537, dtype: int64

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, split, when
from pyspark.sql.types import FloatType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("AmazonReviewDataCleaning") \
    .getOrCreate()

def clean_amazon_reviews(df):
    """
    Function to clean Amazon review data at scale using PySpark.
    Parameters:
        df (DataFrame): Input Spark DataFrame containing Amazon review data.
    Returns:
        DataFrame: Cleaned Spark DataFrame.
    """

    # 1. Drop duplicates
    df = df.dropDuplicates()

    # 2. Handle missing values
    # - Drop reviews with missing text or rating
    df = df.filter(col("text").isNotNull() & col("rating").isNotNull())    

    # - Replace null lists with empty lists in 'categories' and 'features'
    # df = df.withColumn("categories", split(regexp_replace(col("categories").cast("string"), "null", "[]"), ", "))

    # - Handle missing values by replacing NaN with 'unknown', helpful_vote with 0
    df = df.fillna({"title_x": "unknown", "helpful_vote": 0})
    df = df.withColumn("price", 
                       when(col("price").isNull() | (col("price").cast("string") == "NaN"), "unknown")
                       .otherwise(col("price").cast("string")))
    
    # 3. Clean and normalize text columns
    # - Remove extra whitespaces in 'title_x' and 'text' columns
    df = df.withColumn("title_x", regexp_replace(col("title_x"), r"\s+", " "))
    df = df.withColumn("text", regexp_replace(col("text"), r"\s+", " "))

    return df

# df = pd.read_csv('./data/sampled_2018.csv')

col_list = ['rating', 'title_x', 'text','timestamp','helpful_vote','verified_purchase','categories','price','parent_asin']
# Assuming your data is in a Spark DataFrame called `df_raw`
df_raw = spark.createDataFrame(df[col_list].copy())

print("Null values in price before cleaning:", df_raw.filter(col("price").isNull()).count())

df_cleaned = clean_amazon_reviews(df_raw)

print("Null values in price after cleaning:", df_cleaned.filter(col("price") == "unknown").count())
# Show the cleaned data
df_cleaned.show()


24/10/26 18:25:39 WARN TaskSetManager: Stage 21 contains a task of very large size (1166 KiB). The maximum recommended task size is 1000 KiB.


Null values in price before cleaning: 0


24/10/26 18:25:40 WARN TaskSetManager: Stage 24 contains a task of very large size (1166 KiB). The maximum recommended task size is 1000 KiB.


Null values in price after cleaning: 20218


24/10/26 18:25:40 WARN TaskSetManager: Stage 30 contains a task of very large size (1166 KiB). The maximum recommended task size is 1000 KiB.


+------+--------------------+--------------------+--------------------+------------+-----------------+--------------------+-------+-----------+
|rating|             title_x|                text|           timestamp|helpful_vote|verified_purchase|          categories|  price|parent_asin|
+------+--------------------+--------------------+--------------------+------------+-----------------+--------------------+-------+-----------+
|   5.0|Fantastic, qualit...|I have had this b...|2018-01-19 11:29:...|           1|                1|["Home & Kitchen"...|unknown| B01N03RW4P|
|   5.0|          Five Stars|           Thank you|2018-04-23 15:41:...|           0|                1|["Home & Kitchen"...|unknown| B07TKH1T48|
|   3.0| Not for tender feet|It's a very nice ...|2018-10-28 00:13:...|           0|                1|["Home & Kitchen"...|  13.99| B09RYMS3PP|
|   5.0|   Love the quality.|Perfect accent I ...|2018-09-07 16:31:...|           0|                1|["Home & Kitchen"...|  36.27| B073

In [28]:
df_raw.printSchema()

root
 |-- rating: string (nullable = true)
 |-- title_x: string (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- helpful_vote: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- price: string (nullable = true)
 |-- parent_asin: string (nullable = true)



In [37]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, split, when
from pyspark.sql.types import FloatType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("AmazonReviewDataCleaning") \
    .getOrCreate()

def clean_amazon_reviews(df):
    """
    Function to clean Amazon review data at scale using PySpark.
    Parameters:
        df (DataFrame): Input Spark DataFrame containing Amazon review data.
    Returns:
        DataFrame: Cleaned Spark DataFrame.
    """

    # 1. Drop duplicates
    df = df.dropDuplicates()

    # 2. Handle missing values
    # - Drop reviews with missing text or rating
    df = df.filter(col("text").isNotNull() & col("rating").isNotNull())    

    # - Replace null lists with empty lists in 'categories' and 'features'
    df = df.withColumn("categories", split(regexp_replace(col("categories").cast("string"), "null", "[]"), ", "))
    # df = df.withColumn("features", split(regexp_replace(col("features").cast("string"), "null", "[]"), ", "))

    # - Handle missing values by replacing NaN with 'unknown', helpful_vote with 0
    df = df.fillna({"title_x": "unknown", "helpful_vote": 0})
    df = df.withColumn("price", 
                       when(col("price").isNull() | (col("price").cast("string") == "NaN"), "unknown")
                       .otherwise(col("price").cast("string")))
    
    # 3. Clean and normalize text columns
    # - Remove extra whitespaces in 'title_x' and 'text' columns
    df = df.withColumn("title_x", regexp_replace(col("title_x"), r"\s+", " "))
    df = df.withColumn("text", regexp_replace(col("text"), r"\s+", " "))

    return df

# Read the data directly into a Spark DataFrame
df_raw = spark.read.csv(
    './data/sampled_2018.csv',  # Path to your CSV file
    header=True,                # Use the first row as headers
    inferSchema=True,          # Automatically infer the data types
    quote='"',                  # Specify double quotes as the quote character
    escape='"',                 # Escape character for quotes within quoted strings
    # mode='DROPMALFORMED'       # Drop rows that are malformed
)
# Select relevant columns
col_list = ['rating', 'title_x', 'text', 'timestamp', 'helpful_vote', 'verified_purchase', 'categories', 'price', 'parent_asin']
df_raw = df_raw.select(*col_list)

# Print null values in price before cleaning
print("Null values in price before cleaning:", df_raw.filter(col("price").isNull()).count())

# Clean the data
df_cleaned = clean_amazon_reviews(df_raw)

# Print null values in price after cleaning
print("Null values in price after cleaning:", df_cleaned.filter(col("price") == "unknown").count())

# Show the cleaned data
df_cleaned.show()


Null values in price before cleaning: 20218
Null values in price after cleaning: 20207
+------+--------------------+--------------------+--------------------+------------+-----------------+--------------------+-----+-----------+
|rating|             title_x|                text|           timestamp|helpful_vote|verified_purchase|          categories|price|parent_asin|
+------+--------------------+--------------------+--------------------+------------+-----------------+--------------------+-----+-----------+
|   1.0|Not satisfied wit...|I used this rug i...|2018-09-05 17:36:...|           1|                1|[["Home & Kitchen...| 48.0| B0785JHZNF|
|   5.0|        Great value!|The trays are lar...|2018-03-24 19:56:...|           0|                1|[["Home & Kitchen...|38.31| B0BT1H77LH|
|   5.0|          Is worthy.|Great product. Ha...|2018-04-14 01:48:...|           2|                1|[["Home & Kitchen...|29.99| B01HPMRDXC|
|   5.0|                Nice|    Will order again|2018-10-29 

In [39]:
df_cleaned.show()

+------+--------------------+--------------------+--------------------+------------+-----------------+--------------------+-----+-----------+
|rating|             title_x|                text|           timestamp|helpful_vote|verified_purchase|          categories|price|parent_asin|
+------+--------------------+--------------------+--------------------+------------+-----------------+--------------------+-----+-----------+
|   1.0|Not satisfied wit...|I used this rug i...|2018-09-05 17:36:...|           1|                1|[["Home & Kitchen...| 48.0| B0785JHZNF|
|   5.0|        Great value!|The trays are lar...|2018-03-24 19:56:...|           0|                1|[["Home & Kitchen...|38.31| B0BT1H77LH|
|   5.0|          Is worthy.|Great product. Ha...|2018-04-14 01:48:...|           2|                1|[["Home & Kitchen...|29.99| B01HPMRDXC|
|   5.0|                Nice|    Will order again|2018-10-29 06:58:...|           0|                1|[["Home & Kitchen...|18.99| B07D54C8JB|
|   5.

In [79]:
from pyspark.sql.functions import countDistinct
df_cleaned.select(countDistinct("text")).show()

24/10/26 17:16:25 WARN TaskSetManager: Stage 99 contains a task of very large size (2717 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+
|count(DISTINCT text)|
+--------------------+
|               49204|
+--------------------+



In [30]:
df_raw.printSchema()

root
 |-- rating: string (nullable = true)
 |-- title_x: string (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- helpful_vote: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- price: string (nullable = true)
 |-- parent_asin: string (nullable = true)



In [32]:
df_temp = df[col_list].copy()

In [77]:
df_temp['price'].isna().value_counts()

price
False    31717
True     20218
Name: count, dtype: int64

In [44]:
df_temp[['text']].value_counts()

text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
Love it                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [50]:
df_cleaned.count()

24/10/26 16:54:32 WARN TaskSetManager: Stage 48 contains a task of very large size (2717 KiB). The maximum recommended task size is 1000 KiB.


51935

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, split, when

def create_spark_session():
    """
    Create a Spark session.
    Returns:
        SparkSession: Initialized Spark session.
    """
    spark = SparkSession.builder \
        .appName("AmazonReviewDataCleaning") \
        .getOrCreate()
    return spark

def clean_amazon_reviews(spark, file_path):
    """
    Function to clean Amazon review data at scale using PySpark.
    
    Parameters:
        spark (SparkSession): The Spark session.
        file_path (str): Path to the input CSV file.
    
    Returns:
        DataFrame: Cleaned Spark DataFrame.
    """

    # Read CSV as Spark DataFrame
    df_raw = spark.read.csv(file_path, header=True, inferSchema=True)

    # Filter for specific columns
    col_list = ['rating', 'title_x', 'text', 'timestamp', 'helpful_vote', 'verified_purchase', 'categories', 'price', 'parent_asin']
    df_raw = df_raw.select(col_list)

    # 1. Drop duplicates
    df_raw = df_raw.dropDuplicates()

    # 2. Handle missing values
    # - Drop reviews with missing text or rating
    df_raw = df_raw.filter(col("text").isNotNull() & col("rating").isNotNull())    

    # - Replace null lists with empty lists in 'categories'
    # df_raw = df_raw.withColumn("categories", split(regexp_replace(col("categories").cast("string"), "null", "[]"), ", "))

    # - Handle missing values by replacing NaN with 'unknown', helpful_vote with 0
    df_raw = df_raw.fillna({"title_x": "unknown", "helpful_vote": 0})
    # df_raw = df_raw.withColumn("price", 
    #                    when(col("price").isNull() | (col("price").cast("string") == "NaN"), "unknown")
    #                    .otherwise(col("price").cast("string")))
    
    # 3. Clean and normalize text columns
    # - Remove extra whitespaces in 'title_x' and 'text' columns
    df_raw = df_raw.withColumn("title_x", regexp_replace(col("title_x"), r"\s+", " "))
    df_raw = df_raw.withColumn("text", regexp_replace(col("text"), r"\s+", " "))

    return df_raw


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, split, when

# Initialize Spark session
spark = SparkSession.builder \
    .appName("AmazonReviewDataCleaning") \
    .getOrCreate()

def clean_amazon_reviews(df):
    # Remove duplicates and filter out rows with null ratings or texts
    col_list = ['rating', 'title_x', 'text', 'timestamp', 'helpful_vote', 'verified_purchase', 'price', 'parent_asin']
    df = df.select(col_list)


    df = df.dropDuplicates()
    df = df.filter(col("text").isNotNull() & col("rating").isNotNull())
    
    # # Clean categories and features columns
    # df = df.withColumn("categories", 
    #                    split(regexp_replace(col("categories").cast("string"), "null", "[]"), ", "))
    # df = df.withColumn("features", 
    #                    split(regexp_replace(col("features").cast("string"), "null", "[]"), ", "))
    
    # Fill nulls
    df = df.fillna({"title_x": "unknown", "helpful_vote": 0})
    
    # Clean price column
    df = df.withColumn("price", 
                       when(col("price").isNull() | (col("price").cast("string") == "NaN"), "unknown")
                       .otherwise(col("price").cast("string")))
    
    # Clean text and title columns
    df = df.withColumn("title_x", regexp_replace(col("title_x"), r"\s+", " "))
    df = df.withColumn("text", regexp_replace(col("text"), r"\s+", " "))
    
    return df

# Read the data
df_raw = spark.read.csv(
    'staging/data/sampled_2018.csv', 
    header=True, 
    inferSchema=True,
    quote='"',  # Handle quoted strings correctly
    mode='DROPMALFORMED',  # Drop malformed lines
)

# Trim column names
df_raw = df_raw.toDF(*(c.strip() for c in df_raw.columns))

# Clean the data
df_cleaned = clean_amazon_reviews(df_raw)

# Show the cleaned data
df_cleaned.show(truncate=False)

# Optionally convert to Pandas for better readability
df_cleaned_pd = df_cleaned.toPandas()
print(df_cleaned_pd)


+------+---------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------+------------+-----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|rating|title_x                                                  |text                                                                                                                      